# Tutorial for pyfroc

This tutorial demonstrates a typical process of AFROC analysis using pyfroc framework and your DICOM images.

## 1. Install pyfroc

In [ ]:
%%sh
pip install pyfroc

Check if the pyfroc command is installed correctly

In [ ]:
%%sh
pyfroc -h

## 2. Prepare a working directory

In [ ]:
%%sh
mkdir ./sample_experiment
cd ./sample_experiment

## 3. Preapre DICOM images

Prepare your DICOM images on which the reference lesions and responses will be recorded.

All DICOM images should be stored in one directory. Here, we use `dicom` directory.

In [ ]:
%%sh
mkdir ./dicom
cp -r ./your_dicom_data_dir ./dicom

## 4. Prepare directories for this experiment

Run pyfroc command below.

This command generates directories in which the .nrrd file of reference lesions and responses will be stored.

In [1]:
%%sh
pyfroc prepare --dicom-dir ./dicom --output-dir ./experiment --num-of-raters 2 --num-of-modalities 2

usage: Usage: pyfroc [subcommand] [options] prepare [-h] --dicom-dir DICOM_DIR
                                                    [--target-dir TARGET_DIR]
                                                    [--num-of-raters NUM_OF_RATERS]
                                                    [--num-of-modalities NUM_OF_MODALITIES]

options:
  -h, --help            show this help message and exit
  --dicom-dir DICOM_DIR
                        Path to the root directory of DICOM files used in the
                        experiment.
  --target-dir TARGET_DIR
                        Path to the target directory where the prepared files
                        will be stored. The responses of the raters and the
                        reference files should be stored in this directory.
                        The default is ./experiment.
  --num-of-raters NUM_OF_RATERS
                        Number of raters in the experiment. Default is 3.
  --num-of-modalities NUM_OF_MODALITIES
        

The generated directories look like below.

```
experiment
    ├── reference
    │   ├── LIDC-IDRI-0001           <- Patient name
    │   │   └── 20000101_CT         <- Study date and modality
    │   │       └── SE3000566       <- Series number (save .nrrd file here)
    └── responses
        ├── rater01                   <- Rater name
        │   ├── LIDC-IDRI-0001       <- Patient name
        │   │   ├── 20000101_CT0    <- Study date and modality ID
        │   │   │   └── SE3000566  <- Series number (save .nrrd file here)
        │   │   └── 20000101_CT1    <- Study date and modality ID
        │   │       └── SE3000566   <- Series number (save .nrrd file here)
```

## 5. Prepare reference lesions

The built-in loader of pyfroc can read .nrrd files. Here, we use `3D Slicer` to make .nrrd files of the reference lesions. See [3D Slicer image computing platform | 3D Slicer](https://www.slicer.org/) to install 3D Slicer.

After launching 3D Slicer, drag and drop your dicom images from your file exploer to the 3D Slicer window to import.

Then, open the case image and select the segmentation tool.

![img1](./img/tutorial2_1.png)

Annotate one lesion for one segmentation. The lesion is approximated as a minimal sphere enclosing the annotated mask. So you can annotate the lesion only on the largest cross-sectional image. You don't have to annotate entire the lesion.

The confidence level is ignored for the reference lesions. It is used for responses.

![img2](./img/tutorial2_2.png)

Save the segmentations of the reference lesions as a .nrrd file. The file should be in the corresponding `reference` directory prepared before.

![img3](./img/tutorial2_3.png)

If you have more series or patients, repeat the process above.

(*) Remember the "close scene" before starting the next patient's process to prevent contaminating the patients' coordinate systems.

Finally, you have directories and files like below.

![img4](./img/tutorial2_4.png)

## 6. Conduct an image reading experiment for each rater

Make .nrrd files of the responses of each rater in the same way of above.

Remember the .nrrd files should be save the corresponding `responses` directory.

## 7.Evaluate the responses and generate a .xlsx file for AFROC analysis

In [ ]:
%%sh
pyfroc evaluate --eval-dir "./sample_experiment/experiment" --write-img --dicom-dir "./sample_experiment/dicom"


Now, you can find something below.

- `./sample_experiment/experiment/lesion_response_images` directory containing images pairing the responses and reference lesion
- `./sample_experiment/rjafroc_input.xlsx` file for AFROC analysis using RJafroc library

## Run a R script

Run a R script for AFROC and statistical analysis using RJafroc.

Here, we use a sample R script `afroc_analysis.R`. You can find the sciprt at https://github.com/akchan/pyfroc/samples/afroc_analysis.R.

In [ ]:
%%sh
Rscript ./afroc_analysis.R

Finally, you can find the AFROC curve plot as the `wAFROC_averaged.pdf` file and the `wAFROC_each_reader.pdf` file.